In [1]:
#Importing all the necessary libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
import cv2
import os
!pip install gradio
import gradio as gr
# warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
EPOCHS = 25
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((256, 256))
image_size = 0
width=256
height=256
depth=3

In [3]:
# installing and importing the opendatasets so as to use it retrieve datasets from kaggle
!pip install opendatasets --upgrade --quiet
import opendatasets as od

In [7]:
# Downloading the data from kaggle
kaggle_link = 'https://www.kaggle.com/datasets/emmarex/plantdisease'
od.download(kaggle_link, force=True)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: shadrackcheruiyot
Your Kaggle Key: ········


  0%|          | 0.00/658M [00:00<?, ?B/s]

100%|██████████| 658M/658M [1:34:12<00:00, 122kB/s]  


In [10]:
# function used converting images to an array
def imgarray(img_dir):
    try:
        image = cv2.imread(img_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return imgarray(image)
        else :
            return np.array([])
    except Exception as e:
        print(e)
        return None

In [16]:
## In this cell I will be declaring the dataset directory path denoted by "data_dir"
data_dir = "./plantdisease"

In [30]:
from PIL import Image

In [31]:
image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = os.listdir(data_dir)
    for directory in root_dir:
        # remove .DS_Store from list
        if directory == ".DS_Store":
            root_dir.remove(directory)

    for plant_folder in root_dir:
        plant_disease_folder_list = os.listdir(f"{data_dir}/{plant_folder}")
        
        for disease_folder in plant_disease_folder_list:
            # remove .DS_Store from list
            if disease_folder == ".DS_Store":
                plant_disease_folder_list.remove(disease_folder)

        for plant_disease_folder in plant_disease_folder_list:
            print(f"[INFO] Processing {plant_disease_folder} ...")
            plant_disease_image_list = os.listdir(f"{data_dir}/{plant_folder}/{plant_disease_folder}/")
                
            for single_plant_disease_image in plant_disease_image_list:
                if single_plant_disease_image == ".DS_Store":
                    plant_disease_image_list.remove(single_plant_disease_image)

            for image in plant_disease_image_list[:200]:
                image_directory = f"{data_dir}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") or image_directory.endswith(".JPG"):
                    try:
                        img = Image.open(image_directory)
                        image_list.append(img)
                        label_list.append(plant_disease_folder)
                        print("[INFO] Image loading completed")
                    except Exception as e:
                        print(f"Error: {e}")
except Exception as e:
    print(f"Error: {e}")


[INFO] Loading images ...
[INFO] Processing Pepper__bell___Bacterial_spot ...
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading co

[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] I

[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Processing Tomato_Early_blight ...
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading complet

[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] I

[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] I

[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] I

[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Processing Tomato__Tomato_mosaic_virus ...
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading

[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] Image loading completed
[INFO] I

In [37]:
len(image_list)

2952

In [38]:
len(label_list)

2952

In [40]:
# Importing the important libraries for Tranforming image labels
from sklearn.preprocessing import LabelBinarizer
import pickle

In [41]:
## Transforming image Labels using scikit learn (labelBinarizer)
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

In [44]:
# the code below is to show the number of classes of the labels binarized
print(n_classes)

15
